In [52]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/superconductivty-data-data-set/unique_m.csv
/kaggle/input/superconductivty-data-data-set/train.csv


There are two files: (1) train.csv contains 81 features extracted from 21263 superconductors along with the critical temperature in the 82nd column, (2) unique_m.csv contains the chemical formula broken up for all the 21263 superconductors from the train.csv file. The last two columns have the critical temperature and chemical formula. The goal here is to predict the critical temperature based on the features extracted, therefore, it is a univariate regression problem.

In [53]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [54]:
#Get the data
data_train = pd.read_csv('/kaggle/input/superconductivty-data-data-set/train.csv')
data_train.columns

Index(['number_of_elements', 'mean_atomic_mass', 'wtd_mean_atomic_mass',
       'gmean_atomic_mass', 'wtd_gmean_atomic_mass', 'entropy_atomic_mass',
       'wtd_entropy_atomic_mass', 'range_atomic_mass', 'wtd_range_atomic_mass',
       'std_atomic_mass', 'wtd_std_atomic_mass', 'mean_fie', 'wtd_mean_fie',
       'gmean_fie', 'wtd_gmean_fie', 'entropy_fie', 'wtd_entropy_fie',
       'range_fie', 'wtd_range_fie', 'std_fie', 'wtd_std_fie',
       'mean_atomic_radius', 'wtd_mean_atomic_radius', 'gmean_atomic_radius',
       'wtd_gmean_atomic_radius', 'entropy_atomic_radius',
       'wtd_entropy_atomic_radius', 'range_atomic_radius',
       'wtd_range_atomic_radius', 'std_atomic_radius', 'wtd_std_atomic_radius',
       'mean_Density', 'wtd_mean_Density', 'gmean_Density',
       'wtd_gmean_Density', 'entropy_Density', 'wtd_entropy_Density',
       'range_Density', 'wtd_range_Density', 'std_Density', 'wtd_std_Density',
       'mean_ElectronAffinity', 'wtd_mean_ElectronAffinity',
       'gmean_

In [55]:
data_train.head(10)

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.0
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.0
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.0
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.0
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.0
5,4,88.944468,57.795044,66.361592,36.098926,1.181795,1.225203,122.90607,20.687458,51.968828,...,2.214286,2.213364,2.181543,1.368922,1.141474,1,1.000000,0.433013,0.410326,23.0
6,4,88.944468,57.682296,66.361592,36.069470,1.181795,1.316857,122.90607,10.765639,51.968828,...,2.142857,2.213364,2.119268,1.368922,1.194453,1,0.857143,0.433013,0.349927,11.0
7,4,76.517718,57.175142,59.310096,35.891368,1.197273,0.943560,122.90607,36.451199,44.289459,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,33.0
8,4,76.517718,56.808817,59.310096,35.773432,1.197273,0.981880,122.90607,34.833160,44.289459,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,36.0
9,4,76.517718,56.442492,59.310096,35.655884,1.197273,1.016495,122.90607,33.215121,44.289459,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,31.0


In [56]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21263 entries, 0 to 21262
Data columns (total 82 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   number_of_elements               21263 non-null  int64  
 1   mean_atomic_mass                 21263 non-null  float64
 2   wtd_mean_atomic_mass             21263 non-null  float64
 3   gmean_atomic_mass                21263 non-null  float64
 4   wtd_gmean_atomic_mass            21263 non-null  float64
 5   entropy_atomic_mass              21263 non-null  float64
 6   wtd_entropy_atomic_mass          21263 non-null  float64
 7   range_atomic_mass                21263 non-null  float64
 8   wtd_range_atomic_mass            21263 non-null  float64
 9   std_atomic_mass                  21263 non-null  float64
 10  wtd_std_atomic_mass              21263 non-null  float64
 11  mean_fie                         21263 non-null  float64
 12  wtd_mean_fie      

In [57]:
data_train.describe()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
count,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,...,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000
mean,4.115224,87.557631,72.988310,71.290627,58.539916,1.165608,1.063884,115.601251,33.225218,44.391893,...,3.153127,3.056536,3.055885,1.295682,1.052841,2.041010,1.483007,0.839342,0.673987,34.421219
std,1.439295,29.676497,33.490406,31.030272,36.651067,0.364930,0.401423,54.626887,26.967752,20.035430,...,1.191249,1.046257,1.174815,0.393155,0.380291,1.242345,0.978176,0.484676,0.455580,34.254362
min,1.000000,6.941000,6.423452,5.320573,1.960849,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000210
25%,3.000000,72.458076,52.143839,58.041225,35.248990,0.966676,0.775363,78.512902,16.824174,32.890369,...,2.116732,2.279705,2.091251,1.060857,0.775678,1.000000,0.921454,0.451754,0.306892,5.365000
50%,4.000000,84.922750,60.696571,66.361592,39.918385,1.199541,1.146783,122.906070,26.636008,45.123500,...,2.618182,2.615321,2.434057,1.368922,1.166532,2.000000,1.063077,0.800000,0.500000,20.000000
75%,5.000000,100.404410,86.103540,78.116681,73.113234,1.444537,1.359418,154.119320,38.356908,59.322812,...,4.026201,3.727919,3.914868,1.589027,1.330801,3.000000,1.918400,1.200000,1.020436,63.000000
max,9.000000,208.980400,208.980400,208.980400,208.980400,1.983797,1.958203,207.972460,205.589910,101.019700,...,7.000000,7.000000,7.000000,2.141963,1.949739,6.000000,6.992200,3.000000,3.000000,185.000000


In [58]:
data_unique = pd.read_csv('/kaggle/input/superconductivty-data-data-set/unique_m.csv')
data_unique.head(10)

,H,He,Li,Be,B,C,N,O,F,Ne,...,Au,Hg,Tl,Pb,Bi,Po,At,Rn,critical_temp,material
0,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,29.0,Ba0.2La1.8Cu1O4
1,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,26.0,Ba0.1La1.9Ag0.1Cu0.9O4
2,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,19.0,Ba0.1La1.9Cu1O4
3,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,22.0,Ba0.15La1.85Cu1O4
4,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,23.0,Ba0.3La1.7Cu1O4
5,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,23.0,Ba0.5La1.5Cu1O4
6,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,11.0,Ba1La1Cu1O4
7,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,33.0,Sr0.1La1.9Cu1O4
8,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,36.0,Sr0.15La1.85Cu1O4
9,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,31.0,Sr0.2La1.8Cu1O4


In [59]:
#data_unique.info()

Lets try to visualize the data and get some information. A quick way to get a feel of the type of data you are dealing with is to plot a histogram or correlation map for each numerical attribute. However, we have a lot of features (82), so then, it is difficult to understand anything. Hence, we will plot the correlation map only with the target variable.

In [60]:
#Correlation between feature and critical tempeature
correlations = data_train.corr()["critical_temp"].sort_values(ascending=False).drop("critical_temp")
for feature, correlation in correlations.items():
    print(f"{feature} : {correlation:.6f}")

#fig, axes = plt.subplots(nrows=9, ncols=9, figsize=(20, 20))
#axes = axes.flatten()
#for i, feature in enumerate(correlations.index):
#    sns.scatterplot(x=data_train[feature], y=data_train["critical_temp"], ax=axes[i])
#    axes[i].set_title(f'Corr: {correlations[feature]:.2f}')
#    axes[i].set_xlabel(feature)
#    axes[i].set_ylabel("critical_temp")

wtd_std_ThermalConductivity : 0.721271
range_ThermalConductivity : 0.687654
range_atomic_radius : 0.653759
std_ThermalConductivity : 0.653632
wtd_entropy_atomic_mass : 0.626930
wtd_entropy_atomic_radius : 0.603494
number_of_elements : 0.601069
range_fie : 0.600790
wtd_std_atomic_radius : 0.599199
entropy_Valence : 0.598591
wtd_entropy_Valence : 0.589664
wtd_std_fie : 0.582013
entropy_fie : 0.567817
wtd_entropy_FusionHeat : 0.563244
std_atomic_radius : 0.559629
entropy_atomic_radius : 0.558937
entropy_FusionHeat : 0.552709
entropy_atomic_mass : 0.543619
std_fie : 0.541804
range_atomic_mass : 0.491970
wtd_range_ThermalConductivity : 0.469572
entropy_Density : 0.457169
entropy_ElectronAffinity : 0.437207
wtd_entropy_Density : 0.400190
wtd_mean_fie : 0.398796
wtd_entropy_fie : 0.388359
wtd_mean_ThermalConductivity : 0.379336
std_atomic_mass : 0.378766
mean_ThermalConductivity : 0.375813
wtd_std_atomic_mass : 0.359306
wtd_gmean_fie : 0.343747
wtd_std_ElectronAffinity : 0.315147
wtd_range_fi

- The correlation coefficient ranges from –1 to 1. When it is close to 1, it means that there is a strong positive correlation (directly proportional) while the negative sign indicates that the relationship between the variables is inversely proportional. Finally, coefficients close to zero mean that there is no linear correlation. Therefore, at first we cannot discard any attribute. In addiction, if we take a look at the scatterplot, none of the attributes have a completely linear relationship with the target. 

In [61]:
#data_train.plot(kind="scatter", x="mean_Valence", y="critical_temp", alpha=0.1)

In [62]:
y = data_train["critical_temp"]
X = data_train.drop('critical_temp', axis= 1)
print(y.shape)
print(X.shape)

(21263,)
(21263, 81)


- With few exceptions, Machine Learning algorithms don’t perform well when the input numerical attributes have very different scales, so then, one of the most important transformations we need to apply  is feature scaling. Note that "entropy_atomic_mass" has 0.36 as minimun value and 1.98 as a maximun value while for "gmean_atomic_mass" the range is 5 to > 200. 

- Lets try MinMaxScaler: where values are shifted and rescaled so that they end up ranging from 0 to 1.

In [63]:
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)
print(X_norm)

[[0.375      0.40587859 0.25394952 ... 0.15527506 0.14433757 0.14568627]
 [0.5        0.42461131 0.25718675 ... 0.16140434 0.21081851 0.15620209]
 [0.375      0.40587859 0.25406085 ... 0.15936125 0.14433757 0.14823221]
 ...
 [0.125      0.45893123 0.44029915 ... 0.45765281 0.16666667 0.13333333]
 [0.125      0.45893123 0.44763782 ... 0.31606647 0.16666667 0.15416441]
 [0.25       0.39857242 0.39709844 ... 0.25742971 0.47140452 0.5       ]]


In [64]:
x_train, x_test, y_train, y_test = train_test_split(X_norm, y, test_size = 0.20, random_state = 2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(17010, 81)
(4253, 81)
(17010,)
(4253,)


In [65]:
#Select and train a model
linearModel = LinearRegression()
linearModel.fit(x_train, y_train)

LinearRegression()

Just a reminder, Root Mean Square Error (RMSE) is given by

\begin{equation}
RMSE = \left[\frac{1}{m}\sum_{i = 1}^{m} (y_{pred}^{i}  - y_{true}^{i})^{2}\right]^{1/2}
\end{equation}

In [66]:
#Let’s measure this regression model’s RMSE on the whole training set
y_pred_linear = linearModel.predict(x_train)
linear_mse = mean_squared_error(y_train, y_pred_linear)
linear_rmse = np.sqrt(linear_mse)
print(linear_rmse)

17.62091784166398


Okay, this is better than nothing but it not a great score. Critical temperature range is between 0.000210 and 185.0, so a predict error of 17 is not satisfying. Its look like a underfitting case. Lets try another model:

In [67]:
from sklearn.tree import DecisionTreeRegressor
modelDecTree = DecisionTreeRegressor()
modelDecTree.fit(x_train,y_train)

DecisionTreeRegressor()

In [68]:
y_pred_tree = modelDecTree.predict(x_train)
tree_mse = mean_squared_error(y_train, y_pred_tree)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

4.169956434668739


- Despite DecisionTree sound better, lets try to evaluate in a different way. A great alternative is to use Scikit-Learn’s K-fold cross-validation feature. cross_val_score is a crucial function in scikit-learn for evaluating model performance through cross-validation. It splits your data into multiple folds, trains your model on a subset of the data, and evaluates it on the remaining fold. This process is repeated for each fold, providing a more robust estimate of model performance.

- The following code randomly splits the training set into 10 distinct subsets called folds, then it trains and evaluates the Decision Tree model 10 times, picking a different fold for evaluation every time and training on the other 9 folds. The result is an array containing the 10 evaluation scores:

- **scoring: The metric to evaluate the model's performance. Default is 'accuracy' for classification and 'r2' for regression.** 

- $R^{2}$ score: Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse)

In [69]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(modelDecTree, x_train, y_train, cv=10)
tree_rmse_scores = np.sqrt(scores)

In [70]:
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard deviation:", scores.std())

Scores: [0.88504592 0.8782531  0.86086645 0.85920511 0.88302593 0.84812239
 0.86947889 0.85630411 0.89740792 0.87056106]
Mean: 0.8708270880578652
Standard deviation: 0.014405784412219784


In [71]:
scores_linear = cross_val_score(linearModel, x_train, y_train, cv=10)
linear_rmse_scores = np.sqrt(scores_linear)

In [72]:
print("Scores:", scores_linear)
print("Mean:", scores_linear.mean())
print("Standard deviation:", scores_linear.std())

Scores: [0.73193664 0.74790308 0.70006707 0.74014222 0.74032142 0.74343039
 0.7342585  0.70692134 0.73987232 0.74472307]
Mean: 0.7329576047797729
Standard deviation: 0.015451665624078362


- Based on the results, both models look good, with the Decicion Tree model being slightly better.

In [73]:
#Evaluate the model on the test set
final_pred_linear = linearModel.predict(x_test)
final_pred_DecTree = modelDecTree.predict(x_test)

In [75]:
final_linear_mse = mean_squared_error(y_test, final_pred_linear)
final_linear_rmse = np.sqrt(final_linear_mse)
print("RMSE for linear model on test data= ", final_linear_rmse)

RMSE for linear model on test data=  17.344578083438407


In [76]:
final_DecTree_mse = mean_squared_error(y_test, final_pred_DecTree)
final_DecTree_rmse = np.sqrt(final_DecTree_mse)
print("RMSE for Decision Tree model on test set= ", final_DecTree_rmse)

RMSE for Decision Tree model on test set=  12.316154109346865


 - Here we see an overfittin problem in decision tree model. This model works well on the traning set, however, for new datasets it does not have a good prediction. We need to fine-tune it. 
 - Linear model look like "better". 
 
 - Possible ways to reduce this error:

    - eliminate irrelevant attributes, but from the analysis of the data made at the beginning, apparently all of them seem relevant. We can do it using RandomForestRegressor to see the importance of each attribute for making accurate predictions
 
- However, looking at the code of other people who analyzed the same data set, other models like GradientBoostingRegressor, Cat_BOOST and XGB, were used and the results were not that better. They were even used with attribute reduction.